In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import tensorflow as tensor

In [30]:
dataset = pd.read_csv(r'news_articles.tsv', delimiter='\t', quoting=3, encoding='ISO-8859-1')
article = dataset.iloc[:,1]
topic = dataset.iloc[:,2]

In [31]:
master_word_list=[]
master_article=''
for j in range(len(article)):
    master_article+=article[j]

In [69]:
stop_words=set(stopwords.words('english'))
master_word_list=master_article.lower().split()
word_tokens=word_tokenize(master_article.lower())
master_filtered_sentence=[w for w in word_tokens if not w in stop_words]
master_tagged_words=pos_tag(master_filtered_sentence)
master_nouns=[word for word, pos in master_tagged_words\
              if(pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos=='NNPS')]
master_verbs=[word for word, pos in master_tagged_words\
              if(pos=='VB' or pos=='VBD' or pos=='VBG' or pos=='VBN' or pos=='VBP' or pos=='VBZ')]
master_word_freq=[]

for k in range(len(master_filtered_sentence)):
        master_word_freq.append(master_word_list.count(master_filtered_sentence[k]))
idf=[]
for i in range(len(master_filtered_sentence)):
    score=0
    for j in range(len(article)):
        if(master_filtered_sentence[i] in article[j]):
            score+=1
    idf.append(score)    
dict_article={
      'data':master_filtered_sentence,
      'freq':master_word_freq,
      'idf':idf
     }  
columns=['data','freq','idf']     
master_set=pd.DataFrame(dict_article, columns=columns)
master_set=master_set.loc[master_set['freq']!=0]
master_set=master_set.loc[master_set['idf']!=0]
master_set=master_set.drop_duplicates().reset_index()
master_set=master_set.drop(['index'],axis=1)
master_set.iloc[:,1]=master_set.iloc[:,1].values.astype(np.float64)
master_set.iloc[:,2]=master_set.iloc[:,2].values.astype(np.float64)
tot=np.sum(master_set.iloc[:,1])
vocab=master_set.iloc[:,0]
master_set
vocab
if(len(master_nouns)>len(master_verbs)):
    alpha_noun=50
    alpha_verb=20
else:
    alpha_noun=20
    alpha_verb=50
master_set

,data,freq,idf
0,federal,14.0,3.0
1,reserve,7.0,10.0
2,bank,59.0,23.0
3,president,20.0,6.0
4,",",12.0,244.0
5,votes,1.0,1.0
6,policy,19.0,16.0
7,year,44.0,88.0
8,said,241.0,132.0
9,recent,42.0,45.0


In [62]:
final_data=[0]*8382
for l in range (len(article)):
    article_data=article[l]
    stop_words=set(stopwords.words('english'))
    wordlist=article_data.lower().split()
    word_tokens=word_tokenize(article_data.lower())
    nouns = [w for w in word_tokens if not w in stop_words]
    tagged_word=pos_tag(nouns)
    article_nouns=[word for word, pos in tagged_word\
                    if(pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos=='NNPS')]
    article_verbs=[word for word, pos in tagged_word\
                    if(pos=='VB' or pos=='VBD' or pos=='VBG' or pos=='VBN' or pos=='VBP' or pos=='VBZ')]
    word_freq = []
    for i in range(len(nouns)):
        word_freq.append(wordlist.count(nouns[i]))
    dict_data=dict(zip(nouns,word_freq))
    tf=0.0
    sum=0.0
    presence=[]
    tf_idf=0.0
    pmi=0.0
    dict_data={k:v for k,v in dict_data.items() if v!=0}
    for master in range(len(master_set)):
        if(master_set.iloc[master,0] in dict_data):
            tf+=np.log(dict_data.get(master_set.iloc[master,0])/master_set.iloc[master,1])
            #tf+=np.log(master_set.iloc[master,1])
            sum+=dict_data.get(master_set.iloc[master,0])
            #presence.append(1)
        #else:
            #presence.append(0)
    for master in range(len(master_set)):
        if(master_set.iloc[master,0] in dict_data):
            #tf_idf+=(dict_data.get(master_set.iloc[master,0])/sum)*np.log(len(article)/master_set.iloc[master,2])
            if(master_set.iloc[master,0] in article_nouns):
                tf_idf=alpha_noun*(dict_data.get(master_set.iloc[master,0])/sum)*np.log(len(article)/master_set.iloc[master,2])
            elif(master_set.iloc[master,0] in article_verbs):
                tf_idf=alpha_verb*(dict_data.get(master_set.iloc[master,0])/sum)*np.log(len(article)/master_set.iloc[master,2])
            else:
                tf_idf=(dict_data.get(master_set.iloc[master,0])/sum)*np.log(len(article)/master_set.iloc[master,2])
            presence.append(tf_idf)
        else:
            presence.append(0)
    presence.insert(0,np.abs(tf/sum))
    #presence.insert(1,tf_idf/sum)
    #presence.insert(2,pmi)
    final_data=np.column_stack((final_data,presence))
    sum

In [63]:
final_data=final_data[:,1:257]

In [64]:
master_column=master_set.iloc[:,0].tolist()
master_column.insert(0,'tf')
#master_column.insert(1,'tf-idf')
#master_column.insert(2,'pmi')
master_column

['tf',
 'federal',
 'reserve',
 'bank',
 'president',
 ',',
 'votes',
 'policy',
 'year',
 'said',
 'recent',
 'encouraging',
 'economic',
 'data',
 'enough',
 'change',
 'pace',
 'central',
 'asset',
 'purchases',
 '.',
 'would',
 'nice',
 'perspective',
 'started',
 'faster',
 'television',
 'interview',
 'paris',
 'referring',
 'fed',
 'tapering',
 'stimulus',
 'programme',
 'given',
 'fact',
 'embarked',
 'measured',
 'reductions',
 'important',
 'give',
 'certainty',
 'least',
 'clarity',
 'markets',
 'ok',
 'continue',
 '10',
 'billion',
 'hurdle',
 'rate',
 'pretty',
 'high',
 'either',
 'direction',
 'trying',
 'determine',
 'whether',
 'economy',
 'shown',
 'signs',
 'weakness',
 'severe',
 'winter',
 'weather',
 'fundamental',
 'obstacles',
 'growth',
 'policy-setting',
 'open',
 'market',
 'committee',
 '(',
 ')',
 'meets',
 '18-19',
 'march',
 'first',
 'gathering',
 'led',
 'chair',
 'since',
 'succeeded',
 'ben',
 's.',
 'last',
 'c.',
 '7',
 'sees',
 'reasonably',
 'outl

In [65]:
master_data=pd.DataFrame(data=final_data.transpose(),columns=master_column)

In [66]:
master_data

,tf,federal,reserve,bank,president,",",votes,policy,year,said,...,distractingly,repetitive,refining,sessions,repopulated,high-level,ranked,reacting,10th,17th
0,1.786157,0.047054,0.343131,0.509986,1.191561,0.0,0.586791,0.293396,0.000000,0.140185,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1.894590,0.047304,0.689913,0.512699,0.798600,0.0,0.000000,0.000000,0.000000,0.140931,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1.060103,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2.088479,0.067886,0.990105,0.367891,0.573041,0.0,0.000000,0.846592,0.000000,0.101126,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1.417481,0.000000,0.000000,0.905045,0.000000,0.0,0.000000,0.260337,0.000000,0.124390,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,1.362928,0.000000,0.000000,0.406126,0.000000,0.0,0.000000,0.155763,0.000000,0.018606,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,1.456074,0.000000,0.000000,0.507302,0.000000,0.0,0.000000,0.000000,0.056202,0.017431,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,1.771507,0.016408,0.000000,0.711346,0.277005,0.0,0.000000,0.613857,0.236422,0.097768,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,1.881642,0.000000,0.000000,0.944974,0.000000,0.0,0.000000,0.543645,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,1.951314,0.000000,0.000000,1.357568,0.000000,0.0,0.000000,0.000000,0.300800,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [67]:
from sklearn.preprocessing import LabelEncoder
labelEncoder=LabelEncoder()
topic=labelEncoder.fit_transform(topic)

In [68]:
classifier = MultinomialNB(alpha=0.0001)
from sklearn import metrics, cross_validation
predicted=cross_validation.cross_val_predict(classifier, master_data, topic, cv=4)
print (metrics.accuracy_score(topic,predicted))
print (metrics.classification_report(topic,predicted))

0.90234375
             precision    recall  f1-score   support

          0       0.84      0.93      0.88        56
          1       0.93      0.96      0.94        67
          2       0.92      0.79      0.85        61
          3       0.92      0.93      0.92        72

avg / total       0.90      0.90      0.90       256



In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(vocabulary=vocab,norm=None)
X_idf=vectorizer.fit_transform(article)

In [70]:
classifier = MultinomialNB()
from sklearn import metrics, cross_validation
predicted=cross_validation.cross_val_predict(classifier, X_idf, topic, cv=4)
print (metrics.accuracy_score(topic,predicted))
print (metrics.classification_report(topic,predicted))

0.91015625
             precision    recall  f1-score   support

          0       0.86      0.91      0.89        56
          1       0.93      0.97      0.95        67
          2       0.92      0.80      0.86        61
          3       0.92      0.94      0.93        72

avg / total       0.91      0.91      0.91       256

